In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
import math
import matplotlib.pyplot as plt

# =========================================
# 1. 读入数据
# =========================================

trade_file = "TradeData.xlsx"        # 美/巴/阿对华出口数据（国家面板）
annual_file = "TradeDataAnnual.xlsx" # 年度总量 + 关税 + 产量（包括 2025）

df_trade = pd.read_excel(trade_file)
df_annual = pd.read_excel(annual_file)

# 只取需要的列并改名
df_trade = df_trade[['period', 'reporterDesc', 'netWgt', 'primaryValue']].copy()
df_trade = df_trade.rename(columns={
    'period': 'year',
    'reporterDesc': 'country',
    'netWgt': 'weight',
    'primaryValue': 'value'
})
df_annual = df_annual.rename(columns={'period': 'year'})

# 按年份合并年度信息（关税、产量、总进口）到国家数据
df = df_trade.merge(df_annual, on='year', how='left')

# =========================================
# 2. 原始价格 (不再构造含关税价格 eff_price，模型使用实际出口价)
# =========================================

df['price'] = df['value'] / df['weight']

# 避免 log 或比率出错：非正数视为缺失
for col in ['price', 'weight', 'total import wgt']:
    df.loc[df[col] <= 0, col] = np.nan

# =========================================
# 3. 三国产量份额（年度变量）
# =========================================

df['prod_total'] = (
    df['us production'] +
    df['brazil production'] +
    df['argentina production']
)

def prod_for_row(row):
    if row['country'] == 'USA':
        return row['us production']
    elif row['country'] == 'Brazil':
        return row['brazil production']
    elif row['country'] == 'Argentina':
        return row['argentina production']
    else:
        return np.nan

df['prod_i'] = df.apply(prod_for_row, axis=1)
df['prod_share'] = df['prod_i'] / df['prod_total']

# =========================================
# 4. 各国份额 p_i（用重量份额）
# =========================================

df['p_i'] = df['weight'] / df['total import wgt']

# =========================================
# 5. 弹性 elasticity：采用“变化率之比” (ΔW/W_{t-1}) / (ΔP/P_{t-1})
#    与 README 描述的 \partial Weight / \partial Price 一致的离散近似。
#    对价格变化率接近 0 的情况使用 eps 以避免爆炸。
# =========================================

df = df.sort_values(['country', 'year'])

df['weight_prev'] = df.groupby('country')['weight'].shift(1)
df['price_prev'] = df.groupby('country')['price'].shift(1)

df['weight_growth'] = (df['weight'] - df['weight_prev']) / df['weight_prev']
df['price_growth'] = (df['price'] - df['price_prev']) / df['price_prev']

eps_den = 1e-6
mask_small_pg = df['price_growth'].abs() < eps_den
df.loc[mask_small_pg & df['price_growth'].notna(), 'price_growth'] = df.loc[
    mask_small_pg & df['price_growth'].notna(), 'price_growth'
].apply(lambda x: eps_den if x >= 0 else -eps_den)

df['elasticity'] = df['weight_growth'] / df['price_growth']

# =========================================
# 6. ln(p_i / p_US) 作为回归因变量
# =========================================

p_us = df[df['country'] == 'USA'][['year', 'p_i']].rename(columns={'p_i': 'p_us'})
df = df.merge(p_us, on='year', how='left')

df['ln_share_ratio'] = np.log(df['p_i'] / df['p_us'])

# =========================================
# 7. 份额回归（Argentina & Brazil）
#    ln(p_i / p_US) ~ ln(prod_share)
#                     + ln(1+tariff_us-cn) + ln(1+tariff_cn-us)
#                     + elasticity + Brazil dummy
#    去除 trend，使之贴合 README 的重力型结构。
# =========================================

share_df = df[df['country'].isin(['Brazil', 'Argentina'])].copy()

share_df['ln_prod_share'] = np.log(share_df['prod_share'])

# 使用 log(1 + 税率/100)，0 税率 -> ln(1) = 0 合法
share_df['ln_tariff_us_cn'] = np.log1p(share_df['tariff us-cn'] / 10.0)
share_df['ln_tariff_cn_us'] = np.log1p(share_df['tariff cn-us'] / 10.0)

share_df = share_df.replace([np.inf, -np.inf], np.nan)

needed_cols = [
    'ln_share_ratio',
    'ln_prod_share',
    'ln_tariff_us_cn',
    'ln_tariff_cn_us',
    'elasticity'
]
share_df = share_df.dropna(subset=needed_cols)

In [2]:
share_df

,year,country,weight,value,total import wgt,total import value,tariff cn-us,tariff us-cn,us production,brazil production,...,weight_prev,price_prev,weight_growth,price_growth,elasticity,p_us,ln_share_ratio,ln_prod_share,ln_tariff_us_cn,ln_tariff_cn_us
1,2016,Argentina,7.792820e+09,2.792549e+09,8.240790e+10,3.138129e+10,3,3.8,114200000000,114100000000,...,9.723394e+09,0.364097,-0.198549,-0.015786,12.577184,0.437472,-1.531737,-1.610490,0.322083,0.262364
2,2017,Argentina,6.603291e+09,2.414514e+09,9.209009e+10,3.494952e+10,3,3.8,121500000000,119200000000,...,7.792820e+09,0.358349,-0.152644,0.020383,-7.488840,0.344118,-1.568428,-1.645979,0.322083,0.262364
3,2018,Argentina,3.394680e+09,1.313217e+09,8.018668e+10,3.166562e+10,28,12.0,116200000000,117100000000,...,6.603291e+09,0.365653,-0.485911,0.057957,-8.383944,0.102703,-0.886231,-1.671543,0.788457,1.335001
4,2019,Argentina,8.962120e+09,3.012738e+09,8.950158e+10,3.145395e+10,33,12.0,96700000000,126700000000,...,3.394680e+09,0.386845,1.640049,-0.131013,-12.518165,0.252241,-0.923879,-1.785816,0.788457,1.458615
5,2020,Argentina,5.475490e+09,1.867359e+09,1.007798e+11,3.697318e+10,33,12.0,114800000000,133000000000,...,8.962120e+09,0.336163,-0.389041,0.014505,-26.820976,0.344399,-1.846702,-1.801238,0.788457,1.458615
6,2021,Argentina,3.435487e+09,1.781766e+09,9.122557e+10,4.309202e+10,33,19.0,120600000000,152000000000,...,5.475490e+09,0.341040,-0.372570,0.520749,-0.715450,0.299411,-2.073240,-1.993275,1.064711,1.458615
7,2022,Argentina,4.924690e+09,2.891532e+09,8.906077e+10,5.261179e+10,33,19.0,113300000000,154600000000,...,3.435487e+09,0.518636,0.433477,0.132105,3.281310,0.341291,-1.820039,-2.610589,1.064711,1.458615
8,2023,Argentina,1.736410e+09,9.372165e+08,1.028109e+11,5.501865e+10,33,19.0,113300000000,154500000000,...,4.924690e+09,0.587150,-0.647407,-0.080739,8.018475,0.258752,-2.729187,-1.880383,1.064711,1.458615
9,2024,Argentina,3.918799e+09,1.692736e+09,1.035360e+11,4.595399e+10,33,19.0,118800000000,171500000000,...,1.736410e+09,0.539744,1.256840,-0.199708,-6.293397,0.261353,-1.932252,-1.919513,1.064711,1.458615
11,2016,Brazil,3.856391e+10,1.438611e+10,8.240790e+10,3.138129e+10,3,3.8,114200000000,114100000000,...,4.092551e+10,0.385769,-0.057705,-0.032980,1.749684,0.437472,0.067377,-0.916466,0.322083,0.262364


In [3]:
share_df

,year,country,weight,value,total import wgt,total import value,tariff cn-us,tariff us-cn,us production,brazil production,...,weight_prev,price_prev,weight_growth,price_growth,elasticity,p_us,ln_share_ratio,ln_prod_share,ln_tariff_us_cn,ln_tariff_cn_us
1,2016,Argentina,7.792820e+09,2.792549e+09,8.240790e+10,3.138129e+10,3,3.8,114200000000,114100000000,...,9.723394e+09,0.364097,-0.198549,-0.015786,12.577184,0.437472,-1.531737,-1.610490,0.322083,0.262364
2,2017,Argentina,6.603291e+09,2.414514e+09,9.209009e+10,3.494952e+10,3,3.8,121500000000,119200000000,...,7.792820e+09,0.358349,-0.152644,0.020383,-7.488840,0.344118,-1.568428,-1.645979,0.322083,0.262364
3,2018,Argentina,3.394680e+09,1.313217e+09,8.018668e+10,3.166562e+10,28,12.0,116200000000,117100000000,...,6.603291e+09,0.365653,-0.485911,0.057957,-8.383944,0.102703,-0.886231,-1.671543,0.788457,1.335001
4,2019,Argentina,8.962120e+09,3.012738e+09,8.950158e+10,3.145395e+10,33,12.0,96700000000,126700000000,...,3.394680e+09,0.386845,1.640049,-0.131013,-12.518165,0.252241,-0.923879,-1.785816,0.788457,1.458615
5,2020,Argentina,5.475490e+09,1.867359e+09,1.007798e+11,3.697318e+10,33,12.0,114800000000,133000000000,...,8.962120e+09,0.336163,-0.389041,0.014505,-26.820976,0.344399,-1.846702,-1.801238,0.788457,1.458615
6,2021,Argentina,3.435487e+09,1.781766e+09,9.122557e+10,4.309202e+10,33,19.0,120600000000,152000000000,...,5.475490e+09,0.341040,-0.372570,0.520749,-0.715450,0.299411,-2.073240,-1.993275,1.064711,1.458615
7,2022,Argentina,4.924690e+09,2.891532e+09,8.906077e+10,5.261179e+10,33,19.0,113300000000,154600000000,...,3.435487e+09,0.518636,0.433477,0.132105,3.281310,0.341291,-1.820039,-2.610589,1.064711,1.458615
8,2023,Argentina,1.736410e+09,9.372165e+08,1.028109e+11,5.501865e+10,33,19.0,113300000000,154500000000,...,4.924690e+09,0.587150,-0.647407,-0.080739,8.018475,0.258752,-2.729187,-1.880383,1.064711,1.458615
9,2024,Argentina,3.918799e+09,1.692736e+09,1.035360e+11,4.595399e+10,33,19.0,118800000000,171500000000,...,1.736410e+09,0.539744,1.256840,-0.199708,-6.293397,0.261353,-1.932252,-1.919513,1.064711,1.458615
11,2016,Brazil,3.856391e+10,1.438611e+10,8.240790e+10,3.138129e+10,3,3.8,114200000000,114100000000,...,4.092551e+10,0.385769,-0.057705,-0.032980,1.749684,0.437472,0.067377,-0.916466,0.322083,0.262364


In [4]:
share_df.isnull().sum()

year                    0
country                 0
weight                  0
value                   0
total import wgt        0
total import value      0
tariff cn-us            0
tariff us-cn            0
us production           0
brazil production       0
argentina production    0
price                   0
prod_total              0
prod_i                  0
prod_share              0
p_i                     0
weight_prev             0
price_prev              0
weight_growth           0
price_growth            0
elasticity              0
p_us                    0
ln_share_ratio          0
ln_prod_share           0
ln_tariff_us_cn         0
ln_tariff_cn_us         0
dtype: int64

In [5]:
share_df.describe()

,year,weight,value,total import wgt,total import value,tariff cn-us,tariff us-cn,us production,brazil production,argentina production,...,weight_prev,price_prev,weight_growth,price_growth,elasticity,p_us,ln_share_ratio,ln_prod_share,ln_tariff_us_cn,ln_tariff_cn_us
count,18.000000,1.800000e+01,1.800000e+01,1.800000e+01,1.800000e+01,18.000000,18.000000,1.800000e+01,1.800000e+01,1.800000e+01,...,1.800000e+01,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,2020.000000,3.260927e+10,1.397016e+10,9.239992e+10,4.034445e+10,25.777778,13.288889,1.143778e+11,1.380778e+11,4.720556e+10,...,3.117440e+10,0.421899,0.101862,0.030534,-0.145723,0.293527,-0.428624,-1.331129,0.807598,1.179047
std,2.656845,2.931475e+10,1.330196e+10,8.207976e+09,8.944324e+09,12.628425,6.092704,7.090636e+09,1.994155e+10,1.060401e+10,...,2.738170e+10,0.088532,0.579103,0.186676,11.883861,0.089801,1.420561,0.606145,0.294454,0.505719
min,2016.000000,1.736410e+09,9.372165e+08,8.018668e+10,3.138129e+10,3.000000,3.800000,9.670000e+10,1.141000e+11,2.125000e+10,...,1.736410e+09,0.336163,-0.647407,-0.199708,-26.820976,0.102703,-2.729187,-2.610589,0.322083,0.262364
25%,2018.000000,5.062390e+09,2.004148e+09,8.906077e+10,3.166562e+10,28.000000,12.000000,1.133000e+11,1.192000e+11,4.500000e+10,...,5.757440e+09,0.359786,-0.187541,-0.103988,-5.535896,0.258752,-1.757136,-1.797383,0.788457,1.335001
50%,2020.000000,2.376301e+10,8.699426e+09,9.122557e+10,3.697318e+10,33.000000,12.000000,1.148000e+11,1.330000e+11,4.900000e+10,...,2.414365e+10,0.381652,-0.041705,-0.001877,-0.178771,0.299411,-0.409427,-1.263726,0.788457,1.458615
75%,2022.000000,5.984795e+10,2.563187e+10,1.007798e+11,4.595399e+10,33.000000,19.000000,1.188000e+11,1.545000e+11,5.400000e+10,...,5.692186e+10,0.501451,0.357919,0.056513,2.898404,0.344118,0.738603,-0.763670,1.064711,1.458615
max,2024.000000,7.447195e+10,3.891772e+10,1.035360e+11,5.501865e+10,33.000000,19.000000,1.215000e+11,1.715000e+11,5.750000e+10,...,7.447195e+10,0.592639,1.640049,0.520749,32.831033,0.437472,2.119219,-0.626104,1.064711,1.458615


In [6]:
share_df['country'].unique()

array(['Argentina', 'Brazil'], dtype=object)

In [7]:
df.head()

,year,country,weight,value,total import wgt,total import value,tariff cn-us,tariff us-cn,us production,brazil production,...,prod_i,prod_share,p_i,weight_prev,price_prev,weight_growth,price_growth,elasticity,p_us,ln_share_ratio
0,2015,Argentina,9.723394e+09,3.540257e+09,7.796672e+10,2.985114e+10,3,3.8,108800000000,96800000000,...,55500000000,0.212562,0.124712,NaN,NaN,NaN,NaN,NaN,0.350378,-1.033004
1,2016,Argentina,7.792820e+09,2.792549e+09,8.240790e+10,3.138129e+10,3,3.8,114200000000,114100000000,...,57000000000,0.199790,0.094564,9.723394e+09,0.364097,-0.198549,-0.015786,12.577184,0.437472,-1.531737
2,2017,Argentina,6.603291e+09,2.414514e+09,9.209009e+10,3.494952e+10,3,3.8,121500000000,119200000000,...,57500000000,0.192824,0.071705,7.792820e+09,0.358349,-0.152644,0.020383,-7.488840,0.344118,-1.568428
3,2018,Argentina,3.394680e+09,1.313217e+09,8.018668e+10,3.166562e+10,28,12.0,116200000000,117100000000,...,54000000000,0.187957,0.042335,6.603291e+09,0.365653,-0.485911,0.057957,-8.383944,0.102703,-0.886231
4,2019,Argentina,8.962120e+09,3.012738e+09,8.950158e+10,3.145395e+10,33,12.0,96700000000,126700000000,...,45000000000,0.167660,0.100134,3.394680e+09,0.386845,1.640049,-0.131013,-12.518165,0.252241,-0.923879


In [8]:
df['country'].unique()

array(['Argentina', 'Brazil', 'USA'], dtype=object)

In [9]:
X = share_df[['ln_prod_share',
                  'ln_tariff_us_cn',
                  'ln_tariff_cn_us',
                  'elasticity']].copy()
# 国家虚拟变量（基准 Argentina，Brazil = 1）
dummies = pd.get_dummies(share_df['country'])
X = pd.concat([X, dummies], axis=1)

In [10]:
X

,ln_prod_share,ln_tariff_us_cn,ln_tariff_cn_us,elasticity,Argentina,Brazil
1,-1.610490,0.322083,0.262364,12.577184,True,False
2,-1.645979,0.322083,0.262364,-7.488840,True,False
3,-1.671543,0.788457,1.335001,-8.383944,True,False
4,-1.785816,0.788457,1.458615,-12.518165,True,False
5,-1.801238,0.788457,1.458615,-26.820976,True,False
6,-1.993275,1.064711,1.458615,-0.715450,True,False
7,-2.610589,1.064711,1.458615,3.281310,True,False
8,-1.880383,1.064711,1.458615,8.018475,True,False
9,-1.919513,1.064711,1.458615,-6.293397,True,False
11,-0.916466,0.322083,0.262364,1.749684,False,True


In [11]:
corr_features=share_df[['ln_share_ratio','ln_tariff_us_cn', 'ln_tariff_cn_us']]
corr_features.describe()
# calculate correlation
corr_features.corr()

,ln_share_ratio,ln_tariff_us_cn,ln_tariff_cn_us
ln_share_ratio,1.000000,-0.016050,0.056133
ln_tariff_us_cn,-0.016050,1.000000,0.919588
ln_tariff_cn_us,0.056133,0.919588,1.000000
